To create a baseline model using a Language Model (LM) such as a Bigram or Trigram model in Python with TensorFlow, you'll need to use the TensorFlow Datasets library for loading the data, and the TensorFlow text and numpy libraries for preprocessing and constructing the model. Here's an example of creating a Bigram model as a baseline model using TensorFlow:

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# Define the input function for loading and preprocessing the data
def load_and_preprocess():
    # Load the data as a TensorFlow dataset
    data, info = tfds.load('tfds:mldatasets/protein/aa_structure_seqs', with_info=True, shuffle_files=False, as_supervised=True)
    # Filter the data to extract the relevant features and labels
    sequences = data['sequence']
    labels = tf.constant([[label] for label in data['class']])
    # Convert the sequences to lowercase and tokenize them
    sequences = tf.strings.map(lambda x: tf.strings.lower(x), sequences)
    sequences = tf.strings.split(sequences, b" ")
    sequences = tf.stack(sequences, axis=1)
    sequences = tf.cast(sequences, tf.int32)
    # Create a dictionary of unique tokens and their indices
    vocab_size = len(tf.unique(sequences).numpy())
    vocab = {token: i for i, token in enumerate(tf.unique(sequences).numpy())}
    return sequences, labels, vocab, vocab_size

# Load and preprocess the data
sequences, labels, vocab, vocab_size = load_and_preprocess()

# Create the training, validation, and test datasets
batch_size = 32
epochs = 10
train_dataset = tf.data.Dataset.from_tensor_slices((sequences[:int(0.8 * len(sequences))], labels[:int(0.8 * len(labels))])).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((sequences[int(0.8 * len(sequences)):], labels[int(0.8 * len(labels)):])).batch(batch_size)
test_dataset = sequences[int(0.95 * len(sequences)):]

# Create the Bigram model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=5, input_length=None),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128)),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])
model.compile(loss='sparse_categorical_crossent', optimizer='adam', metrics=['accuracy'])

# Create the Bigram model baseline
baseline_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=5, input_length=2),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=False)),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])
baseline_model.compile(loss='sparse_categorical_crossent', optimizer='adam', metrics=['accuracy'])

# Train the models and evaluate their performance
history = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)
baseline_history = baseline_model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)

# Print the performance of the models
print("Model accuracy: {:.2f}".format(history.history['val_accuracy'][-1]))
print("Baseline model accuracy: {:.2f}".format(baseline_history.history['val_accuracy'][-1]))